# CS202: Compiler Construction

## In-class Exercises, Week of 03/27/2023

----

# Tuples and Heap-allocated Data

## Question 1

Describe the differences between heap-allocated and stack-allocated data.

stack   | heap
--------|------
static | dynamic
single 64 bit address | multiple 64 bit addresses
value | structure
finite | unbouded
finite lifetime | infinite lifetime?

- Stack-allocated data has static scope, limited lifetime
- Heap-allocated data lives (potentially) forever
- Heap-allocated data needs to be manually deleted or garbage collected

## Question 2

Draw the heap and root stack at the end of the execution of this program:

```
x = (1, 2)
y = (x, x)
z = (y, x)
```

**Draw your solution on your own paper**

## Question 3

What is the value of `a` in this program?

```
x = (1, 2)
y = (1, x)
z = (y, x)
a = (z[1], 1, z[0])
```

a is a lenght-3 tuple with 3 elements:
- 1st element is a pointer to the same tuple as x
- 2nd element is the number 1
- 3rd element is a pointer to the same tuple as y

## Question 4

Outline the changes to the compiler for implementing tuples.

- typechecker: typecheck tuples
- RCO: tiny chages
- typechecker2
  - record the types of tuple-value variables
    - later, when we assign locations to variables, we need to do something special with tuple-valued variables
  - happens after RCO because RCO creates new temporary variables
  - run it twice so that we catch type errors before messing with the program
- expose-allocation: turn tuple creation into memory allocation and a call to the garbage collector
  - we do this before explicate-control because it involves an if statement
- explicate-control: no changes
- select-instructions: new cases for:
  - collect
  - allocate
  - subscript
  - tuple-set
- allocate-registers
  - when we call the garbage collector, need to spill all tuple-valued variables to the root stack
  - update interference graph to have registers in it
  - update graph coloring to directly assign locations
- patch-instructions: small changes for global variables
- prelude-and-conclusion: initialize the heap and root stack

----

# Typechecker

## Question 5

What are the types of the following expressions?

- `(1, 2, 3)`
- `(1, 2)[1]`
- `(1, (2, 3))`
- `(1, (2, 3))[1]`
- `(1, 2)[1 + 2]`

1. (int, int, int)
2. int 
3. (int, (int, int))
4. (int, int)
5. type error

## Question 6

Describe the typing rules for tuple creation and tuple subscript.

- Type of a tuple is a tuple of types of its elements
- To find the type of a subscript expression, find the type in the subscripted tuple at the given index
- Subscript index must be a Constant expression

## Question 7

Describe the changes to the typechecker. Why do we need to run the typechecker again after RCO?

- Add the new typing rules
- Record the types of tuple-valued variables
- We run the typechecker again after RCO because RCO creates lots of tmp variables

----
# RCO

## Question 8

Describe the changes to RCO.

No changes needed

## Question 9

Write the output of RCO for the following program:

```
x = (1 + 2, 3, (4, 5))
y = (x, 1)[1]
```

In [ ]:
tmp_1 = 1 + 2
tmp_2 = (4,5)
x = (tmp_1, 3, tmp_2)
tmp_3 = (x, 1)
y = tmp_3[1]

- Every tuple construction is of the form:
  `Assign(x, Prim('tuple', atomic_args))`

----

# Expose Allocation

## Question 10

Describe the approach in the expose-allocation pass for a statement:

```
x = (1, 2, 3)
```

- Turn tuple construction into uses of "allocate", "collect" and "tuple-set"
- if the free pointer + amount of space needed for the new tuple is greater than the formspace_end, run the garbage collector
- Allocate enought space for the tuple and assign x to point to the new space (use "allocate")
- Set each index of the tuple to the correct value (use "tuple-set")

## Question 11

Write the output of expose-allocation for the statement:

```
x = (1, 2, 3)
```

```
# 1. call collect if needed
if free_ptr + 32 < fromspace_end:
  pass # don't need to call the collect, there is enough space
else:
  _ = collect(32)

# 2. allocate
x = allocate(32)

# 3. set contents
_ = tutple_set(x, 0, 1)
_ = tutple_set(x, 1, 2)
_ = tutple_set(x, 2, 3)
```

----
# Select Instructions

## Question 12

Write x86 instructions corresponding to the output of `select-instructions` on the following `assign` statement:

```
Assign("x", Prim('allocate', [Constant(24)]))
```

YOUR ANSWER HERE

## Question 13

Write x86 instructions corresponding to the output of `select-instructions` on the following statement:

```
Assign('_', Prim('collect', [Constant(16)]))
```

YOUR ANSWER HERE

## Question 14

Write x86 instructions corresponding to the output of `select-instructions` on the following statements:

a. `Assign('y', Prim('subscript', [Var('x'), Constant(0)]))`

b. `Assign('_', Prim('tuple_set', [Var('x'), Constant(0), Constant(1)]))`

YOUR ANSWER HERE